# RunPod GPU 관리 노트북

REST API 직접 호출 방식 (serverless-mlops 패턴).
GPUType enum으로 타입 안전하게 GPU를 지정하고, pandas DataFrame으로 Pod 목록을 확인한다.

| GPU       | VRAM | ~$/hr | 용도          |
|-----------|------|------|-------------|
| RTX 4090  | 24GB | 0.39 | 2B 모델 학습/추론 |
| L40S      | 48GB | 0.74 | 7B 모델 서빙    |
| A100 80GB | 80GB | 1.64 | 대형 모델       |

In [1]:
import os, sys
from dotenv import load_dotenv

load_dotenv()
sys.path.insert(0, os.path.abspath(".."))

from utils.runpod_client import *

print("RunPod REST 클라이언트 로드 완료")

RunPod REST 클라이언트 로드 완료


In [ ]:
# 셀 2: Pod 목록 조회 → DataFrame
import pandas as pd

pod_list = pods()
if pod_list:
    df = pd.DataFrame(pod_list)
    display(df)
else:
    print("실행 중인 Pod이 없습니다.")

In [ ]:
# 셀 3: Pod 생성 (GPUType enum 사용)
pod_id = create(
    name="my-workspace",
    gpu_id=GPUType.NVIDIA_GEFORCE_RTX_4090,
    gpu_count=1,
    volume=50,
    image_name="runpod/pytorch:2.4.0-py3.11-cuda12.4.1-devel-ubuntu22.04",
)
print(f"생성된 Pod ID: {pod_id}")

In [ ]:
# 셀 4: 특정 Pod 상태 조회
# pod_id = "POD_ID"
pod_info = pod(pod_id)
pod_info

In [ ]:
# 셀 5: Pod 삭제
# pod_id = "POD_ID"
result = delete("6bo9b41ss21l6t")
print(f"삭제 결과: {result}")

In [ ]:
# 셀 6: VLM 서빙 Pod 생성
vlm_pod_id = create_from_template(
    name="qwen3-vl-4b-serving",
    gpu_id=GPUType.NVIDIA_GEFORCE_RTX_5080,
)
print(f"VLM Pod ID: {vlm_pod_id}")

In [3]:
# 셀 7: vLLM 연결 테스트
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

VLLM_BASE_URL = "https://api.runpod.ai/v2/8iiebdj6zt0fbd/openai/v1" # 예: http://your-pod-ip:8000/v1
MODEL_NAME = "qwen/qwen3-vl-4b-thinking-fp8"

if VLLM_BASE_URL != "여기에_URL을_입력하세요":
    llm = ChatOpenAI(
        base_url=VLLM_BASE_URL,
        model=MODEL_NAME,
        temperature=0,
        api_key=os.getenv("RUNPOD_API_KEY")
    )

    response = llm.invoke([
        HumanMessage(content="안녕! 너는 어떤 모델이니? 도구 호출이 가능하니?")
    ])
    print("\n[응답 결과]")
    print(response.content)
else:
    print("VLLM_BASE_URL을 입력해 주세요.")


[응답 결과]
Okay, the user is asking, "안녕! 너는 어떤 모델이니? 도구 호출이 가능하니?" which translates to "Hello! What kind of model are you? Can you call tools?" 

First, I need to identify the user's intent. They want to know my model type and if I can use tools. Since they're asking in Korean, I should respond in Korean as well.

I should start by introducing myself as Qwen, the large language model developed by Tongyi Lab. Mention my capabilities like answering questions, creating text, coding, etc.

Next, the user is asking about tool calling. I need to clarify that while I can't directly call external tools, I can use the tools provided by Alibaba Cloud, like the Tongyi app or other integrated services. Maybe mention that I can help with specific tasks if they provide the necessary information.

Wait, the user might be confused about what "도구 호출" means. In some contexts, tool calling refers to accessing external APIs or services. I should explain that while I can't call arbitrary tools, I can use th

In [4]:
# 셀 8: vLLM Tool Calling 디버그
# LangChain bind_tools vs raw HTTP 비교로 tool call 미생성 원인 파악
import requests as _requests, json as _json
from langchain_core.tools import tool
from pydantic import BaseModel, Field

# ── 1) Tool 정의 (move 하나만 간단히) ──────────────────────────
class Action(BaseModel):
    direction: str = Field(description="이동 방향", enum=["UP", "DOWN", "LEFT", "RIGHT"])
    steps: int = Field(description="이동 칸수 (1~3)")

class MoveArgs(BaseModel):
    actions: list[Action] = Field(description="최대 4개 행동을 순서대로 실행")

@tool(args_schema=MoveArgs)
def move(actions: list[Action]) -> str:
    """플레이어를 이동시킨다."""
    return "ok"

TEST_PROMPT = "너는 격자 맵 위의 플레이어야. 현재 (0,0)에 있고 목표는 (2,1)이야. move 도구를 사용해서 이동해."

# ── 2) LangChain bind_tools 방식 ──────────────────────────────
print("=" * 60)
print("[LangChain bind_tools 방식]")
print("=" * 60)

llm_with_tools = llm.bind_tools([move])
lc_resp = llm_with_tools.invoke(TEST_PROMPT)

print(f"\n📌 type        : {type(lc_resp).__name__}")
print(f"📌 content     :\n{lc_resp.content[:500] if lc_resp.content else '(empty)'}")
print(f"📌 tool_calls  : {lc_resp.tool_calls}")
print(f"📌 additional_kwargs : {_json.dumps(lc_resp.additional_kwargs, indent=2, ensure_ascii=False)}")

# ── 3) Raw HTTP 방식 (OpenAI format) ─────────────────────────
print("\n" + "=" * 60)
print("[Raw HTTP 방식 — vLLM 직접 호출]")
print("=" * 60)

raw_tools = [
    {
        "type": "function",
        "function": {
            "name": "move",
            "description": "플레이어를 이동시킨다. 최대 4개 행동을 순서대로 실행.",
            "parameters": {
                "type": "object",
                "properties": {
                    "actions": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "direction": {"type": "string", "enum": ["UP", "DOWN", "LEFT", "RIGHT"]},
                                "steps": {"type": "integer"},
                            },
                            "required": ["direction", "steps"],
                        },
                    }
                },
                "required": ["actions"],
            },
        },
    }
]

payload = {
    "model": MODEL_NAME,
    "messages": [{"role": "user", "content": TEST_PROMPT}],
    "tools": raw_tools,
    "temperature": 0,
}

raw_resp = _requests.post(
    f"{VLLM_BASE_URL}/chat/completions",
    headers={"Authorization": f"Bearer {os.getenv('RUNPOD_API_KEY')}"},
    json=payload,
    timeout=120,
)

print(f"\n📌 status_code : {raw_resp.status_code}")
raw_json = raw_resp.json()
print(f"📌 raw JSON    :\n{_json.dumps(raw_json, indent=2, ensure_ascii=False)}")

# ── 4) 요약 비교 ─────────────────────────────────────────────
print("\n" + "=" * 60)
print("[비교 요약]")
print("=" * 60)
lc_has_tool = bool(lc_resp.tool_calls)
raw_choices = raw_json.get("choices", [{}])
raw_has_tool = bool(raw_choices[0].get("message", {}).get("tool_calls"))
print(f"  LangChain tool_calls 존재: {lc_has_tool}")
print(f"  Raw HTTP  tool_calls 존재: {raw_has_tool}")
if raw_has_tool and not lc_has_tool:
    print("  → vLLM은 tool call을 생성하지만 LangChain 파싱에서 누락됨 (LangChain 이슈)")
elif not raw_has_tool and not lc_has_tool:
    print("  → vLLM 서버 자체가 tool call을 생성하지 않음 (모델/서버 설정 이슈)")
elif raw_has_tool and lc_has_tool:
    print("  → 양쪽 모두 정상 동작")


C:\Users\hun41\AppData\Local\Temp\ipykernel_27744\2607549183.py:9: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'enum'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  direction: str = Field(description="이동 방향", enum=["UP", "DOWN", "LEFT", "RIGHT"])


[LangChain bind_tools 방식]

📌 type        : AIMessage
📌 content     :
Okay, let's see. The user is a player on a grid map starting at (0,0) and needs to get to (2,1). They want me to use the move tool to help.

First, I need to figure out the direction and steps required. The current position is (0,0), target is (2,1). So, the x-coordinate needs to go from 0 to 2, which is a difference of +2. The y-coordinate needs to go from 0 to 1, which is +1.

But the move function allows up to 4 actions, each with direction (UP, DOWN, LEFT, RIGHT) and steps (1-3). So, maybe I
📌 tool_calls  : [{'name': 'move', 'args': {'actions': [{'direction': 'RIGHT', 'steps': 2}, {'direction': 'UP', 'steps': 1}]}, 'id': 'chatcmpl-tool-8d81501f9185445b', 'type': 'tool_call'}]
📌 additional_kwargs : {
  "refusal": null
}

[Raw HTTP 방식 — vLLM 직접 호출]

📌 status_code : 200
📌 raw JSON    :
{
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "ann

In [5]:
# 셀 9: vLLM Tool Calling 디버그 — 이미지(multimodal) 입력 테스트
# 가설: 이미지를 함께 보내면 tool calling이 깨지는지 확인
# cell-8의 llm, VLLM_BASE_URL, MODEL_NAME, raw_tools 재사용
import base64 as _base64, io as _io
from PIL import Image as _Image, ImageDraw as _ImageDraw

# ── 1) 간단한 10x10 격자 PNG 생성 → base64 data URL ──────────
img = _Image.new("RGB", (100, 100), "white")
draw = _ImageDraw.Draw(img)
for i in range(0, 101, 10):
    draw.line([(i, 0), (i, 100)], fill="black", width=1)
    draw.line([(0, i), (100, i)], fill="black", width=1)
# 플레이어(0,0)=빨간 점, 목표(2,1)=초록 점
draw.ellipse([2, 82, 8, 88], fill="red")      # (0,0) → 왼쪽 하단
draw.ellipse([22, 72, 28, 78], fill="green")   # (2,1) → 오른쪽 위
buf = _io.BytesIO()
img.save(buf, format="PNG")
b64_str = _base64.b64encode(buf.getvalue()).decode()
image_data_url = f"data:image/png;base64,{b64_str}"
print(f"✅ 테스트 이미지 생성 완료 (base64 길이: {len(b64_str)})")

# ── 2) LangChain bind_tools + 이미지 ─────────────────────────
print("\n" + "=" * 60)
print("[LangChain bind_tools + 이미지]")
print("=" * 60)

from langchain_core.messages import HumanMessage as _HM

lc_msg = _HM(content=[
    {"type": "text", "text": TEST_PROMPT},
    {"type": "image_url", "image_url": {"url": image_data_url}},
])

llm_with_tools = llm.bind_tools([move])

try:
    lc_img_resp = llm_with_tools.invoke([lc_msg])
    print(f"\n📌 type        : {type(lc_img_resp).__name__}")
    print(f"📌 content     :\n{lc_img_resp.content[:500] if lc_img_resp.content else '(empty)'}")
    print(f"📌 tool_calls  : {lc_img_resp.tool_calls}")
    lc_img_has_tool = bool(lc_img_resp.tool_calls)
except Exception as e:
    print(f"\n❌ LangChain 에러: {e}")
    lc_img_has_tool = False

# ── 3) Raw HTTP + 이미지 (사파리 에이전트 동일 구조) ──────────
print("\n" + "=" * 60)
print("[Raw HTTP + 이미지 — 사파리 에이전트 동일 구조]")
print("=" * 60)

payload_img = {
    "model": MODEL_NAME,
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": TEST_PROMPT},
                {"type": "image_url", "image_url": {"url": image_data_url}},
            ],
        }
    ],
    "tools": raw_tools,
    "temperature": 0,
}

try:
    raw_img_resp = _requests.post(
        f"{VLLM_BASE_URL}/chat/completions",
        headers={"Authorization": f"Bearer {os.getenv('RUNPOD_API_KEY')}"},
        json=payload_img,
        timeout=120,
    )
    print(f"\n📌 status_code : {raw_img_resp.status_code}")
    raw_img_json = raw_img_resp.json()
    print(f"📌 raw JSON    :\n{_json.dumps(raw_img_json, indent=2, ensure_ascii=False)}")
    raw_img_choices = raw_img_json.get("choices", [{}])
    raw_img_has_tool = bool(raw_img_choices[0].get("message", {}).get("tool_calls"))
except Exception as e:
    print(f"\n❌ Raw HTTP 에러: {e}")
    raw_img_json = {}
    raw_img_has_tool = False

# ── 4) 비교 요약: 텍스트 전용 vs 이미지 포함 ─────────────────
print("\n" + "=" * 60)
print("[비교 요약: 텍스트 전용 (cell-8) vs 이미지 포함 (cell-9)]")
print("=" * 60)
print(f"  {'방식':<30} {'텍스트만':>10} {'텍스트+이미지':>14}")
print(f"  {'-'*30} {'-'*10} {'-'*14}")
print(f"  {'LangChain bind_tools':<30} {'O':>10} {'O' if lc_img_has_tool else 'X':>14}")
print(f"  {'Raw HTTP':<30} {'O':>10} {'O' if raw_img_has_tool else 'X':>14}")

if not lc_img_has_tool and not raw_img_has_tool:
    print("\n  🔴 결론: 이미지 입력이 tool calling을 방해한다!")
    print("  → 사파리 에이전트에서 tool call이 안 되는 원인은 이미지(multimodal) 입력")
elif lc_img_has_tool and raw_img_has_tool:
    print("\n  🟢 결론: 이미지 포함해도 tool calling 정상 동작")
    print("  → 이미지가 원인이 아님, 다른 변수(프롬프트/메시지 구조 등) 조사 필요")
else:
    print(f"\n  🟡 결론: 부분적 차이 발견 — 추가 조사 필요")
    print(f"  → LangChain: {'정상' if lc_img_has_tool else '실패'}")
    print(f"  → Raw HTTP: {'정상' if raw_img_has_tool else '실패'}")

✅ 테스트 이미지 생성 완료 (base64 길이: 512)

[LangChain bind_tools + 이미지]

📌 type        : AIMessage
📌 content     :
Okay, let's see. The user is on a grid map, starting at (0,0), and needs to get to (2,1). The current position is (0,0), which is the bottom-left corner maybe? Wait, in grid coordinates, sometimes (0,0) is top-left, but the image shows a red dot at the bottom left and green at (2,1) maybe? Wait, the user says current position is (0,0), target is (2,1). So the grid is probably 0-based.

First, I need to figure out the directions. The user is at (0,0), target is (2,1). So they need to move right 2
📌 tool_calls  : [{'name': 'move', 'args': {'actions': [{'direction': 'RIGHT', 'steps': 2}, {'direction': 'UP', 'steps': 1}]}, 'id': 'chatcmpl-tool-98fcd48f971a63f1', 'type': 'tool_call'}]

[Raw HTTP + 이미지 — 사파리 에이전트 동일 구조]

📌 status_code : 200
📌 raw JSON    :
{
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "ann